In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 5)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import re
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import dash
import dash_table

### Dash Pivottable資料前處理

In [ ]:
data=[
            ['Animal', 'Count', 'Location'],
            ['Zebra', 5, 'SF Zoo'],
            ['Tiger', 3, 'SF Zoo'],
            ['Zebra', 2, 'LA Zoo'],
            ['Tiger', 4, 'LA Zoo'],
        ]
x = pd.DataFrame(data, columns=['Animal', 'Count', 'Location'])

In [ ]:
sales = pd.read_excel('../Dashboard/Weekly report_v0.1.xlsx', sheet_name='出貨明細')

In [ ]:
sales['BG'] = sales.apply(lambda x: 'RFBU2' if 'RFDP' in x['品名'] else 'RFBU1' if 'RFDP' not in x['品名'] and 'RF' in x['BG'] else x['BG'], axis=1)

In [ ]:
sales = sales[sales['狀態'].str.contains('出')]

In [ ]:
sales = sales[['BG', 'Subcategory','Group', '銷售單號', '開單日期', '預交日期', '預交年份', '預交月份', '負責業務', '產品分類', '品名', '幣別', '單價', '數量', '本國幣別NTD', '客戶料號', 'Term']]

In [ ]:
mon_dict = {'January':1, 'February':2, 'March':3, 'April':4, 'May':5, 'June':6, 'July':7, 'August':8, 'September':9, 'October':10, 'November':11, 'December':12}

In [ ]:
sales['數量'] = sales['數量'].astype('int')
sales['本國幣別NTD'] = sales['本國幣別NTD'].astype('int')
sales['預交月份'] = sales['預交月份'].map(mon_dict)


In [ ]:
sales['Subcategory'] = sales['產品分類'].str.upper().str.extract('([A-Z]+)')

In [ ]:
sales = sales[['BG', 'Subcategory', 'Group', '銷售單號', '開單日期', '預交日期', '預交年份', '預交月份',
       '負責業務', '品名', '幣別', '單價', '數量', '本國幣別NTD', '客戶料號', 'Term']]


In [ ]:
mon_order=['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
y['預交月份'] = pd.Categorical(y['預交月份'], ordered=True, categories=mon_order)

In [ ]:
m = sales.groupby(['品名'])[['單價']].mean().reset_index()

In [ ]:
dict_test = [{'label': i, 'value': j} for i, j in m.values]

In [ ]:
sales[['品名', '單價']].values

In [ ]:
m.to_dict('records')

In [ ]:
price_table = sales[['Group', '開單日期', '負責業務', '品名', '客戶料號', '幣別', '單價', '數量']]
price_table['單價'] = price_table.apply(lambda x: x['單價'] /30 if x['幣別'] == 'NTD' else x['單價'] / 6.9 if x['幣別'] == 'CNY' 
                                     else x['單價'] * 1.19 if x['幣別'] == 'EUR' else x['單價'], axis=1)
price_table['幣別'] = price_table['幣別'].apply(lambda x: 'USD')

price_table = price_table.drop_duplicates(subset=['Group', '負責業務', '品名'], keep='last')

app = JupyterDash(__name__)
app.layout = html.Div([
    html.H3('報價記錄'),
    dcc.Dropdown(
    id='test',
    options=[
        {'label':i, 'value': i}
        for i in sales['品名'].unique()
    ],
    value= '', placeholder='Select or enter a PN'
    ),
    html.Div([
        dcc.Graph(id='price_table', figure={}),
        dcc.Graph(id='price_box', figure={})
    ]
    )
])

@app.callback(
[dash.dependencies.Output('price_table', 'figure'), dash.dependencies.Output('price_box', 'figure')],
[dash.dependencies.Input('test', 'value')])

def update_table(item):
    dff = price_table[price_table['品名'] == item]
    fig1 = go.Figure(data=[go.Table(
    header=dict(values=list(dff.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[dff.Group, dff.開單日期, dff.負責業務, dff.品名, dff.客戶料號, dff.幣別, dff.單價, dff.數量],
               fill_color='lavender',
               align='left',
              format = [None, None, None, None, None, None, ",.4f", None]))
])
    fig2 = px.box(dff, x='負責業務', y='單價')
    fig2.update_traces(quartilemethod="exclusive")
    
    return fig1, fig2


if __name__ == '__main__':
    app.run_server(mode='inline')
    

In [ ]:
data = y.values.tolist()

In [ ]:
data.insert(0, y.columns.tolist())

In [ ]:
for index, row in y.iterrows():
    print(row)

### ERP歷史數據

In [ ]:
df = pd.read_excel(r"C:\Users\kaihsu\Desktop\業績總表\歷史數據\訂單出貨明細_axmr4301_2016.xlsx", usecols='A:G,I,O,N,Q,R,S,T,U,V,Y,AC,AE,AF')
df1 = pd.read_excel(r"C:\Users\kaihsu\Desktop\業績總表\歷史數據\訂單出貨明細_axmr4301_2017.xlsx", usecols='A:G,I,O,N,Q,R,S,T,U,V,Y,AC,AE,AF')
df2 = pd.read_excel(r"C:\Users\kaihsu\Desktop\業績總表\歷史數據\訂單出貨明細_axmr4301_2018.xlsx", usecols='A:G,I,O,N,Q,R,S,T,U,V,Y,AC,AE,AF')
df3 = pd.read_excel(r"C:\Users\kaihsu\Desktop\業績總表\歷史數據\訂單出貨明細_axmr4301_2019.xlsx", usecols='A:G,I,O,N,Q,R,S,T,U,V,Y,AC,AE,AF')
df4 = pd.read_excel(r"C:\Users\kaihsu\Desktop\業績總表\歷史數據\訂單出貨明細_axmr4301_2020.xlsx", usecols='A:G,I,O,N,Q,R,S,T,U,V,Y,AC,AE,AF')

In [ ]:
sales = pd.concat([df, df1, df2, df3, df4], axis=0, ignore_index=True)

In [ ]:
sales = sales.query('單位 == "KPCS" | 單位 == "PCS"')

In [ ]:
sales['數量'] = sales.apply(lambda x: x['數量'] * 1000 if x['單位'] == 'KPCS' else x['數量'], axis=1)

In [ ]:
sales['單價'] = sales.apply(lambda x: x['單價'] / 1000 if x['單位'] == 'KPCS' else x['單價'], axis=1)

In [ ]:
sales['單價'] = sales.apply(lambda x: x['單價'] / 30 if x['幣別'] == 'NTD' else x['單價'] / 6.9 if x['幣別'] == 'CNY'
                                     else x['單價'] * 1.19 if x['幣別'] == 'EUR' else x['單價'], axis=1)

In [ ]:
sales['幣別'] = sales['幣別'].apply(lambda x: 'USD')

In [ ]:
price_table = sales[['客戶名稱', '開單日期', '組別', '訂單業務', '品名', '客戶料號', '幣別', '單價', '數量']]

In [ ]:
price_table.drop_duplicates(subset=['客戶名稱', '訂單業務', '品名'], keep='last')

In [ ]:
price_table.to_csv(r"C:\Users\kaihsu\Desktop\his_price.csv", index=False)

### 多個檔案寫入一個excel檔案，但是在不同的sheets

In [ ]:
pip install openpyxl

In [ ]:
writer = pd.ExcelWriter(r'C:\Users\kaihsu\Desktop\差異比較表.xlsx')
weekly.to_excel(writer, sheet_name='CS提供的數據', index=False)
system.to_excel(writer, sheet_name='系統撈出來的數據', index=False)
writer.save()

### MT-System Energomera報價


In [ ]:
mt = pd.read_excel(r"D:\客戶\Mt System\價格表\20201119_MT-System_Energomera_Package.xlsx")

In [ ]:
mt.columns

In [ ]:
mt['EAU in number'] = mt.apply(lambda x: int(x['EAU'].split(' ')[0]) * 1000, axis=1)

In [ ]:
mt['Target Price in USD'] = mt.apply(lambda x: float(x['Target'].split(' ')[0].replace(',', '.')), axis=1)

In [ ]:
mt = mt.reindex(columns=['Competitor PN',
 'INPAQ PN',
 'Link',
 'EAU',
 'EAU in number',
 'Buying lot',
 'Target',
 'Target Price in USD',
 'Defect rate in as-delivered condition (exceeding which the entire batch is returned), %',
 'Warranty period(Year)',
 'Number of failures during the warranty period, %',
 'Service life of a component, year',
 'Shelf life - before installation in the product, year',
 'Reliability of supplied components and materials (failure rate), Hours(FIT/MTBF)'])

In [ ]:
mt.to_excel(r'C:\Users\kaihsu\Desktop\energomera.xlsx', index=False)